# Retinotopy Lab

`Author:  Dan Mossing, Biophysics PhD student in the Adesnik Lab at UC Berkeley`

Topographic maps seem to be a fundamental organizing principle of primary sensory cortex. In this lab, we will examine the retinotopic organization of neurons in primary visual cortex based on calcium imaging data.

Here, we have transgenically labeled somatostatin-expressing interneurons with the fluorescent calcium reporter GCaMP6s, and used two photon imaging to record several planes at varying depths in layer 2/3 through a cranial window. Meanwhile, a monitor shows small, isolated patches of drifting black and white gratings in various locations, against a gray screen.

As in many sensory neuroscience experiments, the stimulation computer here delivers known stimuli, precisely timed and randomly interleaved. A TTL pulse from the stimulation computer to the acquisition computer allows us to record the precise timing of the stimuli relative to the recorded neural activity. We start by using an array of inferred "event rates" computed from the raw fluorescence data, an array of stimulus parameters, and a vector of stimulus times. 

In [ ]:
import matplotlib
matplotlib.use('nbAgg')

In [ ]:
import scipy.ndimage.measurements as snm
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed, HTML
import ipywidgets as widgets
import sklearn.linear_model
import sklearn.cross_validation

In [ ]:
# load the data necessary for this lab
np_file = np.load("data/retinotopy_files.npz")

event_rate = np_file['event_rate'] # the (N,T) event rate data, where N is the number of cells, and T is the number of frames in the experiment

max_projection = np_file['max_projection'] # a max intensity projection of the data, in z and in time
# anterior is left-to-right, medial is bottom-to-top

pix_um = np_file['pix_um'] # width of an imaging pixel, in microns (um)

stim_frame = np_file['stim_frame'] # a (K,2) vector indicating onset and offset of each stimulus

stim_location = np_file['stim_location'] # a (K,2) vector indicating (elevation,azimuth) of each stimulus

depth = np_file['depth'] # a (N,) vector indicating the plane of each ROI. Lower numbers are deeper planes, and the planes are separated by 50 um

sq_deg = np_file['sq_deg'] # the interval at which visual stimuli tile the monitor, in visual degrees

imaging_Hz = np_file['imaging_Hz'] # rate at which a given plane is sampled, in Hz

In [ ]:
# This data is large, had to be saved into two files to avoid github errors
roi_mask = np.vstack((np.load("data/roi_mask_pt1.npy"), np.load("data/roi_mask_pt2.npy")))
# This holds binary masks of the N segmented ROIs

First, let's look at the picture of all of our cells.

In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(max_projection)
plt.show()

And now, overlaying the segmented ROIs:

In [ ]:
plt.figure(figsize=(8,6))
overlay = np.zeros(max_projection.shape+(3,))
overlay[:,:,0] = roi_mask.max(0)
overlay[:,:,1] = max_projection/max_projection.max()
plt.imshow(overlay)
plt.show()

Now, examining some event rate traces at random to get a sense for our data.  Feel free the run the next cell multiple times to get a sense for the diversity in firing rate among the cells that were imaged.

In [ ]:
(N,T) = event_rate.shape

plt.figure(figsize=(10,2))
t = np.arange(T)/imaging_Hz
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.plot(t,event_rate[np.random.choice(N)])
plt.subplot(1,5,1)
plt.xlabel('t (sec)')
plt.ylabel('event rate (a.u.)')
plt.show()

Now for some science. We'll first split up our continuous data into a series of sweeps, centered around the time of each stimulus onset. 

In [ ]:
nbefore = 2 # we will use a couple frames before each stim onset as a measure of "baseline"
nafter = 0

In [ ]:
# write a function to split the (N,T) continuous time series into traces
# of duration Ttrial to yield an array of size (N,K,Ttrial)
K = stim_frame.shape[0]
stim_duration = np.diff(stim_frame,axis=1)
mean_stim_duration = int(np.round(np.mean(stim_duration)))
Ttrial = mean_stim_duration+nbefore+nafter
trial_event_rate = np.zeros((N,K,Ttrial))
for roi in range(N):
    for trial in range(K):
        trial_event_rate[roi,trial,:] = event_rate[roi,stim_frame[trial,0]-nbefore:
                                                       stim_frame[trial,0]+mean_stim_duration+nafter]

Then we'll plot what each neuron tends to do on average, across all different kinds of trials..

In [ ]:
t = np.arange(-nbefore,mean_stim_duration+nafter)/imaging_Hz
plt.plot(t,trial_event_rate.mean(1).T,alpha=0.1)
plt.plot((0,0),(0,1),c='r')
plt.xlabel('Time (s)')
plt.ylabel('Event Rate (a.u.)')
plt.show()

#### Do you notice anything about when the majority of neurons become most active relative to the stimulus?

Next, we'll further split up the data by stimulus condition.  This is so that we can look at the responses of neurons to different kinds of stimuli in order to infer their receptive fields.

In [ ]:
# based on the position (Px,Py) of the stimulus, split the array further into an array of size (N,NPx,NPy,Nreps,Ttrial)
(NPy,NPx) = stim_location.max(0)+1
Nreps = int(K/NPy/NPx)
response = np.zeros((N,NPy,NPx,Nreps,Ttrial))
for roi in range(N):
    for Py in range(NPy):
        for Px in range(NPx):
            look_at = np.logical_and(Py==stim_location[:,0],Px==stim_location[:,1])
            response[roi,Py,Px,:,:] = trial_event_rate[roi,look_at,:]

In [ ]:
# average over the 'Nreps' and 'Ttrial' column to yield retinotopic maps
rf = response[:,:,:,:,nbefore:Ttrial-nafter].mean(-1).mean(-1)

Now that we have averaged the responses across repetitions, we have an estimate of the receptive field for each neuron.  The following function will allow us to look at the receptive fields for all of our neurons at once.

In [ ]:
# this function shows a series of arrays in rows of 15
def imshow_in_rows(arr,rowlen=15,scale=0.5):
    nrows = np.ceil(arr.shape[0]/rowlen)
    plt.figure(figsize=(scale*rowlen,scale*nrows))
    for k in range(arr.shape[0]):
        plt.subplot(nrows,rowlen,k+1)
        plt.imshow(arr[k])
        plt.axis('off')

In [ ]:
# This cell make take several seconds to run, be patient
imshow_in_rows(rf)
plt.show()

This information on its own isn't that informative though.  You can see all of the receptive fields, but we cannot tell which cell each one belongs to.

Use the following code to explore the receptive fields of the cells.  If you click on a cell in the left subplot, you should see it become highlighted, and then the right plot will show that cell's receptive field.  See if you think there is an underlying relationship between the spatial location of the cells and their receptive fields, and write down some of your observations below.

In [ ]:
fig = plt.figure(figsize=(10,3))
ax = fig.add_subplot(1,2,1)
mask_data = roi_mask.max(0)
im = ax.imshow(mask_data, vmax=2)
text = ax.text(0, 0, "", va="bottom", ha="left")

ax2 = fig.add_subplot(1,2,2)
im2 = ax2.imshow(np.zeros_like(rf[0]), vmax=0.3)


def onclick(event):
    cell_n = np.where(roi_mask[:, int(event.ydata), int(event.xdata)])[0][0]
    tx = 'cell number=%d' % (cell_n)
    text.set_text(tx)
    
    mask_data = roi_mask.max(0)
    mask_data[roi_mask[cell_n]==1] = 2
    im.set_data(mask_data)
    
    im2.set_data(rf[cell_n])
    im2.set_clim([0,rf[cell_n].max()])
    fig.canvas.draw()
       
cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()

#### Share some of your findings here.  Do you notice anything surprising about the data?  

In our data, we have both stimulus-driven and stimulus-suppressed somatostatin neurons. We don't expect the center of mass metric to work well for the suppressive RF of the latter group, so we'll ignore them for now.

In [ ]:
spont = trial_event_rate[:,:,:2].mean(-1).mean(-1)
evoked = rf.mean(-1).mean(-1)
_ = plt.scatter(spont,evoked,s=5)
_ = plt.plot((0,0.4),(0,0.4),'r')
_ = plt.xlabel('spontaneous event rate')
_ = plt.ylabel('stimulus-evoked event rate')
plt.show()

#### We only want the cells that have a higher evoked event rate than spontaneous event rate.  Are those cells below or above the shown diagonal line in the figure?

In [ ]:
look_at = evoked > spont  

The question that we ultimately want to answer is:  Do the receptive field locations of these somatostatin neurons follow a topographic map? 

To investigate this quantitatively, we'll compute the ROI centers and receptive field centers using a simple metric, center of mass. Fortunately, Python has lots of libraries for things like this, so we don't have to write much ourselves here.

In [ ]:
rf_comy,rf_comx = zip(*[snm.center_of_mass(r) for r in rf])
rf_comy = rf_comy*sq_deg
rf_comx = rf_comx*sq_deg

In [ ]:
roi_mask_comy,roi_mask_comx = zip(*[snm.center_of_mass(r) for r in roi_mask])
roi_mask_comy = roi_mask_comy*pix_um
roi_mask_comx = roi_mask_comx*pix_um

Next, we're going to visually inspect our cells to see if they seem to follow a retinotopic map. We'll color the ROIs according to their azimuth in our first set of plots, and then their elevation in the next set.  Also, remember that we are looking at cells of varying depths, so we will show each of those in a different subplot to avoid problems with plotting overlapping cells.

In [ ]:
plt.figure(figsize=(10,3))
for k in range(4):
    plt.subplot(1,4,k+1)
    filt = np.logical_and(look_at,depth==k)
    plt.imshow(np.nansum(roi_mask[filt]*rf_comx[filt][:,np.newaxis,np.newaxis],axis=0))
    plt.title("Depth {}".format(k+1))
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
for k in range(4):
    plt.subplot(1,4,k+1)
    filt = np.logical_and(look_at,depth==k)
    plt.imshow(np.nansum(roi_mask[filt]*rf_comy[filt][:,np.newaxis,np.newaxis],axis=0))
    plt.title("Depth {}".format(k+1))
plt.show()

#### From these plots, do you think that the receptive field centers are varying smoothly across the cells that we have imaged?  Why or why not?  If you see any outliers, describe them.

Next, we can plot x-y location on the brain against x-y receptive field location to get a better idea.

In [ ]:
plt.figure(figsize=(9,9))
plt.subplot(2,2,1)
plt.scatter(roi_mask_comx[look_at],rf_comx[look_at])
plt.xlabel('rostral distance (um)')
plt.ylabel('visual azimuth (deg)')

plt.subplot(2,2,2)
plt.scatter(roi_mask_comx[look_at],rf_comy[look_at])
plt.xlabel('rostral distance (um)')
plt.ylabel('visual elevation (deg)')

plt.subplot(2,2,3)
plt.scatter(roi_mask_comy[look_at],rf_comx[look_at])
plt.xlabel('lateral distance (um)')
plt.ylabel('visual azimuth (deg)')

plt.subplot(2,2,4)
plt.scatter(roi_mask_comy[look_at],rf_comy[look_at])
plt.xlabel('lateral distance (um)')
_ = plt.ylabel('visual elevation (deg)')

plt.show()

#### Do these plots match the intuition that you had for the previous question?  Explain what you think these plots are showing.

Now, let's look at whether we can predict receptive field centers from location in the brain. We will use cross-validation, meaning that we will use part of our data to estimate the axes of the retinotopic map, and part of our data to test how accurately these retinotopic axes predict receptive field centers.

In [ ]:
regr = sklearn.linear_model.LinearRegression()
X = np.concatenate((roi_mask_comx[look_at,np.newaxis],roi_mask_comy[look_at,np.newaxis]),axis=1)
y = np.concatenate((rf_comx[look_at,np.newaxis],rf_comy[look_at,np.newaxis]),axis=1)
predicted = sklearn.model_selection.cross_val_predict(regr,X,y)
score = sklearn.model_selection.cross_val_score(regr,X,y)

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.scatter(y[:,0],predicted[:,0])
plt.plot(y[:,0],y[:,0],c='r')
plt.ylabel("RF center")
plt.xlabel("X-axis location in brain")

plt.subplot(1,2,2)
plt.scatter(y[:,1],predicted[:,1])
plt.plot(y[:,1],y[:,1],c='r')
plt.ylabel("RF center")
plt.xlabel("Y-axis location in brain")

plt.show()

In [ ]:
# Here are the cross-validated R^2 values for each resampling
print(score)

#### How do you feel about these $R^2$ values that we got from this dataset?  If you are not familiar with $R^2$, take a quick look at [this wikipedia page](https://en.wikipedia.org/wiki/Coefficient_of_determination). Do you think that the location of cells does a good job of predicting the receptive field centers?